In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import re


/Users/polinab/.virtualenvs/base/lib/python3.8/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
data = pd.read_csv('./data/Consumidor_Venta_Producto_UPC_Recom_2018_2020.csv')
english_cols = {'FACTURA_ID': 'INVOICE_ID', 'FACTURA_POSICION_ID': 'INVOICE_POSITION_ID', 'CUSTOMER_ID': 'CUSTOMER_ID',
                'FACTURA_CLASE_DOCUMENTO_ID': 'INVOICE_DOCUMENT_CLASS_ID', 'ANO_MES_FACTURA': 'INVOICE_MONTH_YEAR',
                'ANO_FACTURA': 'ORDER_YEAR', 'MES_FACTURA': 'ORDER_MONTH', 'FECHA_FACTURA': 'INVOICE_DATE',
                'IMP_VENTA_NETO_EUR': 'NET_SALES_EUR', 'CANAL_VENTA_ID': 'SALES_CHANNEL_ID',
                'CANAL_VENTA_DESC': 'SALES_CHANNEL_DESC', 'TEMPORADA_COMERCIAL_ID': 'SEASON_ID',
                'TEMPORADA_COMERCIAL_DESC': 'SEASON_DESC', 'PRODUCTO_ID': 'PRODUCT_ID', 'TALLA': 'SIZE',
                'MATERIAL_ID': 'MATERIAL_ID', 'NUMERO_DEUDOR_PAIS_ID': 'SALE_COUNTRY_ID',
                'NUMERO_DEUDOR_PAIS_DESC': 'SALE_COUNTRY_DESC', 'VENTA_DEVOLUCION': 'SALES_FILTER',
                'JERARQUIA_PROD_ID': 'PRODUCT_HIERARCHY_ID', 'GRUPO_ARTICULO_PRODUCTO_ID': 'PRODUCT_GROUP_ID',
                'GRUPO_ARTICULO': 'PRODUCT_GROUP', 'CONCEPTO': 'CONCEPT', 'LINEA': 'LINE',
                'GENERO_PRODUCTO': 'PRODUCT_GENDER', 'CATEGORIA': 'PRODUCT_CATEGORY', 'TIPOLOGIA': 'PRODUCT_TYPE',
                'COLOR': 'COLOR_INTERN', 'CONSUMER_COLOR': 'COLOR_EXTERN', 'CREMALLERA': 'ZIPPER', 'CORDONES': 'LACES',
                'OUTSOLE_SUELA_TIPO': 'SOLE_TYPE', 'OUTSOLE_SUELA_SUBTIPO': 'SOLE_SUBTYPE',
                'PLANTILLA_EXTRAIBLE': 'REMOVABLE_SOLE', 'CONTACTO_SN': 'PHYSICAL_CONTACT', 'EDAD_SN': 'AGE_AVAILABLE',
                'GENERO_CONTACTO': 'GENDER_AVAILABLE', 'EDAD_COMPRA': 'AGE_AT_PURCHASE',
                'EDAD_RANGO_COMPRA': 'AGE_RANGE',
                'PAIS_CONTACTO': 'COUNTRY_CONTACT_ID', 'PAIS_CONTACTO_DESC': 'COUNTRY_CONTACT_DESC',
                'CIUDAD_CONTACTO': 'CITY_CONTACT', 'IDIOMA_CONTACTO': 'LANGUAGE_CONTACT'}


In [3]:
dropcols = ['INVOICE_MONTH_YEAR', 'ORDER_YEAR','ORDER_MONTH','SALES_CHANNEL_ID',\
            'SALES_CHANNEL_DESC','AGE_AVAILABLE','MATERIAL_ID','COUNTRY_CONTACT_DESC',\
            'SALE_COUNTRY_DESC','SALES_FILTER','SEASON_ID','GENDER_AVAILABLE']

In [4]:
# gets most common country for the code
def country_dict(df, id_field,desc_field):
    country_dict = {x[id_field]:x[desc_field] for x in df.groupby(id_field)[desc_field].apply(lambda x: x.mode()).to_frame().reset_index().to_dict('index').values()}
    return country_dict

In [5]:
country_lookup = country_dict(data, 'PAIS_CONTACTO','PAIS_CONTACTO_DESC')
country_lookup.update(country_dict(data, 'NUMERO_DEUDOR_PAIS_ID','NUMERO_DEUDOR_PAIS_DESC'))
country_lookup



{'AD': 'ANDORRA',
 'AE': 'EMIRATOS ÁRABES UNIDOS',
 'AF': 'AFGANISTÁN',
 'AG': 'ANTIGUA Y BARBUDA',
 'AL': 'ALBANIA',
 'AR': 'ARGENTINA',
 'AT': 'AUSTRIA',
 'AU': 'AUSTRALIA',
 'AZ': 'AZERBAYÁN',
 'BA': 'BOSNIA-HERZEGOVINA',
 'BE': 'BÉLGICA',
 'BG': 'BULGARIA',
 'BR': 'BRAZIL',
 'BY': 'BIELORRUSIA',
 'CA': 'CANADA',
 'CH': 'SWITZERLAND',
 'CL': 'CHILE',
 'CN': 'CHINA',
 'CO': 'COLOMBIA',
 'CR': 'COSTA RICA',
 'CU': 'CUBA',
 'CY': 'CHIPRE',
 'CZ': 'REPÚBLICA CHECA',
 'DE': 'GERMANY',
 'DK': 'DENMARK',
 'EC': 'ECUADOR',
 'EE': 'ESTONIA',
 'EG': 'EGIPTO',
 'ES': 'SPAIN',
 'FI': 'FINLANDIA',
 'FR': 'FRANCE',
 'GB': 'UNITED KINGDOM OF GREAT BRITAIN AND NORTHERN IRELAND',
 'GE': 'GEORGIA',
 'GP': 'GUADALUPE',
 'GR': 'GREECE',
 'HK': 'HONG KONG',
 'HR': 'CROACIA',
 'HU': 'HUNGRÍA',
 'IE': 'IRELAND',
 'IL': 'ISRAEL',
 'IN': 'INDIA',
 'IS': 'ISLANDIA',
 'IT': 'ITALY',
 'JP': 'JAPAN',
 'KR': 'KOREA REPUBLIC OF',
 'KZ': 'KAZAJSTÁN',
 'LA': 'LAOS',
 'LI': 'LIECHTENSTEIN',
 'LT': 'LITUANIA',
 'LU':

In [6]:
def preprocess(d):
    d_copy = d.rename(columns=english_cols)
    d_copy['REMOVABLE_SOLE'] = d_copy['REMOVABLE_SOLE'].apply(lambda x: True if x == 'Extraible' else False)
    
    invoice_id_dict = {'ZTON':'Sale','ZDVN':'Return','ZDAN':'Cancelled Return'}
    d_copy['INVOICE_DOCUMENT_CLASS_ID'] = d_copy['INVOICE_DOCUMENT_CLASS_ID'].apply(lambda x: invoice_id_dict[x])
    
    d_copy['INVOICE_DATE'] = pd.to_datetime(d_copy.INVOICE_DATE)
    
    d_copy.AGE_AT_PURCHASE = d_copy.AGE_AT_PURCHASE.replace(0, np.nan)
    
    d_copy.ZIPPER = d_copy.ZIPPER.apply(lambda x: True if x in ('SI','YES') else False)
    d_copy.LACES = d_copy.LACES.apply(lambda x: True if x in ('With laces', 'Con cordones') else False)
    
    product_group_dict = {'Zapatos Adulto':'Adult Shoes',
                          'Bolsos':'Bag',
                          'Bolsos Cartujano':'Bag',
                          'Ropa':'Clothings',
                          'Complementos':'Complements'
                         }
    d_copy.PRODUCT_GROUP = d_copy.PRODUCT_GROUP.apply(lambda x: product_group_dict[x] if x in product_group_dict else x)
    
    color_dict = {'Мульти ассорти':'Multi - Assorted',
                  'красный': 'Red',
                  'розовый' : 'Pink',
                  'желтый' : 'Yellow'
                 }
    d_copy.COLOR_INTERN = d_copy.COLOR_INTERN.apply(lambda x: color_dict[x] if x in color_dict else x)
    
    d_copy.PHYSICAL_CONTACT = d_copy.PHYSICAL_CONTACT.apply(lambda x: True if x == 'S' else False)
    
    d_copy.SEASON_DESC = d_copy.SEASON_DESC.apply(lambda x: "".join(re.split('^\d{2}|\W+', x)))
    
    d_copy.SOLE_SUBTYPE = d_copy.SOLE_SUBTYPE.apply(lambda x: np.nan if pd.isnull(x) else "".join(re.split('\(.*\)', x)))
    
    
    return d_copy



In [7]:
cleandata = preprocess(data)
cleandata = cleandata.drop(columns=dropcols)



In [8]:
item_fields = ['PRODUCT_GROUP','CONCEPT','LINE',\
               'PRODUCT_GENDER','PRODUCT_CATEGORY','PRODUCT_TYPE',\
               'LACES','ZIPPER','SOLE_TYPE','SOLE_SUBTYPE','REMOVABLE_SOLE','SEASON_DESC',\
              'COLOR_INTERN']

item_bools = ['LACES','ZIPPER','REMOVABLE_SOLE']

item_categoricals = ['PRODUCT_GROUP','CONCEPT','LINE','PRODUCT_GENDER',\
                     'PRODUCT_CATEGORY','PRODUCT_TYPE','SOLE_TYPE',\
                     'SOLE_SUBTYPE','SEASON_DESC','COLOR_INTERN'
                    ]


In [46]:
cleandata[cleandata.PRODUCT_ID == '15233-001']

,INVOICE_ID,INVOICE_POSITION_ID,CUSTOMER_ID,INVOICE_DOCUMENT_CLASS_ID,INVOICE_DATE,NET_SALES_EUR,SEASON_DESC,PRODUCT_ID,SIZE,SALE_COUNTRY_ID,...,LACES,SOLE_TYPE,SOLE_SUBTYPE,REMOVABLE_SOLE,PHYSICAL_CONTACT,AGE_AT_PURCHASE,AGE_RANGE,COUNTRY_CONTACT_ID,CITY_CONTACT,LANGUAGE_CONTACT
3348,2944641540,1,nc83ag23YFsfmvlxWTSVOqn5P3V4c8kPlqGGIA5ilpI=,Sale,2019-11-29,220.29,FW2019,15233-001,47,US,...,True,FLAT,FLAT,False,True,NaN,NS,US,Friday Harbor,EN
26234,2963606892,1,Uf96QkRxHWCVALCTm8fUhs7l9FnhKc82VwCadtSUgC4=,Sale,2019-12-09,164.54,FW2019,15233-001,46,PL,...,True,FLAT,FLAT,False,True,53.0,50-59,PL,kalisz,PL
27964,2911687687,1,gal5Uj26Ieyg75TOjZFaV7uPhNI9ZUK5MN8Mc19bqYY=,Sale,2019-12-10,175.63,FW2019,15233-001,47,ES,...,True,FLAT,FLAT,False,True,NaN,NS,ES,LEÓN,ES
38220,2945656359,1,9v3RNggx9UG5pVPB7bdks2goksdpxSJ3XLdcqAMLTfw=,Sale,2019-12-18,170.16,FW2019,15233-001,44,DE,...,True,FLAT,FLAT,False,True,NaN,NS,DE,Hofgeismar,DE
40417,2911689432,1,6rSzeypOsFrETzd+ub48xmXxBYrf1p/qmnEugNKVTrU=,Sale,2019-12-20,175.63,FW2019,15233-001,43,ES,...,True,FLAT,FLAT,False,True,31.0,30-39,ES,Málaga,ES
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1316996,2911636171,1,vkDlZGhFADKs3npHszcwhs/MZ2vi9FNSzStOYgU1gX0=,Sale,2019-06-06,170.45,SS2019,15233-001,41,ES,...,True,FLAT,FLAT,False,True,NaN,NS,ES,LA HERRERA,ES
1321583,2011712624,1,R2WdIHve7+8/RYHwQLqiEVUDt8DEjHnOJhfnslvBXRc=,Sale,2020-12-29,175.63,FW2020,15233-001,43,ES,...,True,FLAT,FLAT,False,True,43.0,40-49,ES,Santa Perpètua de Mogoda,ES
1323764,2011712986,1,Thavc1tEURQGMMccsUPyJdWFf5EfROCcY2y0/lvcd70=,Sale,2020-12-30,179.55,FW2020,15233-001,43,DK,...,True,FLAT,FLAT,False,True,51.0,50-59,DK,Horsens,EN
1326468,2045685710,1,Uc/cHRHtmSYTcAgLbTdvbvclr9JnKKK4rcLKeouvFLQ=,Sale,2020-12-31,192.89,FW2020,15233-001,47,DE,...,True,FLAT,FLAT,False,True,50.0,50-59,DE,Mülheim an der Ruhr,DE


In [47]:
items_attributes = cleandata[['PRODUCT_ID']+item_fields].drop_duplicates().dropna(subset=['PRODUCT_ID'])
items_attributes[items_attributes.PRODUCT_ID == '15233-001']


,PRODUCT_ID,PRODUCT_GROUP,CONCEPT,LINE,PRODUCT_GENDER,PRODUCT_CATEGORY,PRODUCT_TYPE,LACES,ZIPPER,SOLE_TYPE,SOLE_SUBTYPE,REMOVABLE_SOLE,SEASON_DESC,COLOR_INTERN
3348,15233-001,Adult Shoes,Nautico,Nautico,MEN,Men Mocasin/Nautical,Boat Shoe,True,False,FLAT,FLAT,False,FW2019,Medium Brown
70663,15233-001,Adult Shoes,Nautico,Nautico,MEN,Men Mocasin/Nautical,Boat Shoe,True,False,FLAT,FLAT,False,FW2017,Medium Brown
138218,15233-001,Adult Shoes,Nautico,Nautico,MEN,Men Mocasin/Nautical,Boat Shoe,True,False,FLAT,FLAT,False,SS2019,Medium Brown
232028,15233-001,Adult Shoes,Nautico,Nautico,MEN,Men Mocasin/Nautical,Boat Shoe,True,False,FLAT,FLAT,False,SS2020,Medium Brown
292865,15233-001,Adult Shoes,Nautico,Nautico,MEN,Men Mocasin/Nautical,Boat Shoe,True,False,FLAT,FLAT,False,SS2018,Medium Brown
496067,15233-001,Adult Shoes,Nautico,Nautico,MEN,Men Mocasin/Nautical,Boat Shoe,True,False,FLAT,FLAT,False,FW2018,Medium Brown
970101,15233-001,Adult Shoes,Nautico,Nautico,MEN,Men Mocasin/Nautical,Boat Shoe,True,False,FLAT,FLAT,False,FW2020,Medium Brown


In [11]:
items_attributes.shape

(24760, 14)

In [12]:

onehot_categoricals = pd.get_dummies(items_attributes[item_categoricals])


In [13]:
item_mtx_df = pd.concat([items_attributes.PRODUCT_ID, items_attributes[item_bools]*1, onehot_categoricals], axis=1)


In [14]:
item_mtx_df

,PRODUCT_ID,LACES,ZIPPER,REMOVABLE_SOLE,PRODUCT_GROUP_Adult Shoes,PRODUCT_GROUP_Bag,PRODUCT_GROUP_Clothings,PRODUCT_GROUP_Complements,PRODUCT_GROUP_Kid's Shoes,PRODUCT_GROUP_Kids,...,COLOR_INTERN_Multi - Assorted,COLOR_INTERN_Navy,COLOR_INTERN_No Color,COLOR_INTERN_Nude,COLOR_INTERN_Pink,COLOR_INTERN_Red,COLOR_INTERN_Rust/Copper,COLOR_INTERN_Turquiose Aqua,COLOR_INTERN_White Natural,COLOR_INTERN_Yellow
0,K400208-001,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,36587-052,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,K200734-007,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,K300143-003,1,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,K300287-002,1,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1326173,K900128-001,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1326213,K100718-001,1,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1326706,K200564-018,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1327027,K201146-003,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
item_mtx_df

,PRODUCT_ID,LACES,ZIPPER,REMOVABLE_SOLE,PRODUCT_GROUP_Adult Shoes,PRODUCT_GROUP_Bag,PRODUCT_GROUP_Clothings,PRODUCT_GROUP_Complements,PRODUCT_GROUP_Kid's Shoes,PRODUCT_GROUP_Kids,...,COLOR_INTERN_Multi - Assorted,COLOR_INTERN_Navy,COLOR_INTERN_No Color,COLOR_INTERN_Nude,COLOR_INTERN_Pink,COLOR_INTERN_Red,COLOR_INTERN_Rust/Copper,COLOR_INTERN_Turquiose Aqua,COLOR_INTERN_White Natural,COLOR_INTERN_Yellow
0,K400208-001,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,36587-052,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,K200734-007,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,K300143-003,1,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,K300287-002,1,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1326173,K900128-001,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1326213,K100718-001,1,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1326706,K200564-018,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1327027,K201146-003,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
item_mtx_np = np.asmatrix(item_mtx_df)

In [17]:
item_mtx_np.shape

(24760, 942)

In [31]:
customer_product_lookup = cleandata[['CUSTOMER_ID','PRODUCT_ID']].drop_duplicates().dropna(subset=['CUSTOMER_ID'])

In [32]:
customer_product_lookup.sort_values('CUSTOMER_ID')

,CUSTOMER_ID,PRODUCT_ID
75154,++/DcYFzQ1gIUJza+neA8NadxET0/nofxMP2xeocbt4=,K200313-004
75578,++/DcYFzQ1gIUJza+neA8NadxET0/nofxMP2xeocbt4=,K200289-001
1115536,++/RB4do2DxW6/bkh8jUK3zw9JxzStn/dbEjQxvIxkQ=,K400534-001
1030712,++00bzGka2b8Wztp2jO3FgPtNBlG7EddjjW/7+WxP4o=,K300192-005
700602,++02Ly6o8yS/HLJrSkkNiguWcOWl7268zltL3qvkv4Q=,18302-074
...,...,...
985102,zzx8SoQJefRX1/2jmKTTmFxHk7CbpBe5KV6fOedaSR8=,K300255-002
209681,zzx8SoQJefRX1/2jmKTTmFxHk7CbpBe5KV6fOedaSR8=,K100500-005
348379,zzzQiUfgIUY1DgB653FbVcRUVxKTPafMP2wLDVT8Xzs=,K300005-016
85326,zzzfi+QNP+Ce5s+UcQTCYVDtjLnQtBt6DcCVDI6LvHU=,46104-090


In [33]:
merged_customer_product = customer_product_lookup.merge(item_mtx_df, on='PRODUCT_ID')


In [34]:
merged_customer_product.sort_values('CUSTOMER_ID')

,CUSTOMER_ID,PRODUCT_ID,LACES,ZIPPER,REMOVABLE_SOLE,PRODUCT_GROUP_Adult Shoes,PRODUCT_GROUP_Bag,PRODUCT_GROUP_Clothings,PRODUCT_GROUP_Complements,PRODUCT_GROUP_Kid's Shoes,...,COLOR_INTERN_Multi - Assorted,COLOR_INTERN_Navy,COLOR_INTERN_No Color,COLOR_INTERN_Nude,COLOR_INTERN_Pink,COLOR_INTERN_Red,COLOR_INTERN_Rust/Copper,COLOR_INTERN_Turquiose Aqua,COLOR_INTERN_White Natural,COLOR_INTERN_Yellow
3064873,++/DcYFzQ1gIUJza+neA8NadxET0/nofxMP2xeocbt4=,K200289-001,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1720681,++/DcYFzQ1gIUJza+neA8NadxET0/nofxMP2xeocbt4=,K200313-004,1,0,1,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3064872,++/DcYFzQ1gIUJza+neA8NadxET0/nofxMP2xeocbt4=,K200289-001,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3064871,++/DcYFzQ1gIUJza+neA8NadxET0/nofxMP2xeocbt4=,K200289-001,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1720682,++/DcYFzQ1gIUJza+neA8NadxET0/nofxMP2xeocbt4=,K200313-004,1,0,1,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2866309,zzzfi+QNP+Ce5s+UcQTCYVDtjLnQtBt6DcCVDI6LvHU=,46104-090,1,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2866308,zzzfi+QNP+Ce5s+UcQTCYVDtjLnQtBt6DcCVDI6LvHU=,46104-090,1,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2866307,zzzfi+QNP+Ce5s+UcQTCYVDtjLnQtBt6DcCVDI6LvHU=,46104-090,1,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3994106,zzzfi+QNP+Ce5s+UcQTCYVDtjLnQtBt6DcCVDI6LvHU=,K200441-012,1,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [36]:
user_mtx_df = merged_customer_product.groupby('CUSTOMER_ID').max().drop(columns=['PRODUCT_ID'])

In [43]:
user_mtx_np = np.asmatrix(user_mtx_df)

In [44]:
user_mtx_np.shape

(516919, 941)